In [2]:
import pandas as pd
import numpy as np
import os

In [ ]:
'''
- data_supplied is all data from user

- data_full is data full infomation

- data_from_t24 is data extra from t24

- data_account_from_t24 is name account from t24

'''

# # import data

In [ ]:
# import data
def import_data(folder):
    data_supplied = []
    for file in os.listdir(folder):
        df = pd.read_excel(os.path.join(folder, file), dtype=object)
        data_supplied.append(df)
        data_supplied = pd.concat(data_supplied)

    return data_supplied

data_supplied = import_data('fileDataSupplied')



#  # transform data supplied

In [ ]:
# Remove duplicate columns ID
data_supplied.drop_duplicates('ID', inplace=True)

# remove comma
def processFT(x): 
    # Take first 12 character if is FT

    if x.startswith("FT"):
        x = x[:12]
        return x
    else:
        return x

data_supplied['ID'] = data_supplied['ID'].apply(processFT)


# # Export file SQL

In [ ]:
dict_data ={}


for index, row in data_supplied.iterrows():

    if row['ID'].startswith('FT'):
        if row['TKNO'] is np.NaN or row['TKNO'].startswith(('VND','PL')): 

            if row['TKCO'] in dict_data:
                dict_data[row['TKCO']] = dict_data[row['TKCO']] + f",'{row['ID'][:12]}'"
    
            else:
                dict_data[row['TKCO']] = f"'{row['ID'][:12]}'"
        else:

            if row['TKNO'] in dict_data:
                dict_data[row['TKNO']] = dict_data[row['TKNO']] + f",'{row['ID'][:12]}'"
            else:
                dict_data[row['TKNO']] = f"'{row['ID'][:12]}'"


# dict_data
for key, value in dict_data.items():
    account = key
    list_ft = value
    count = len(value.split(","))
    if len(value.split(",")) < 1000:
    # account =key
    # list_ft = value
        with open(os.path.join('OCBDWP'+ f"//{account}.txt"), mode='w', encoding= "utf-8-sig") as file:
            file.write(f'''
-- OCBDWP
select ft_no trans_no
, ben_acct_no tai_khoan_doi_ung
, ben_cust_name ten_doi_ung
, ben_bank_name ngan_hang_doi_ung
from ocbdwp.v_cust_acct_stmt_ben
where account = '{account}'
and ft_no in (
{list_ft}
)
and ben_acct_no is not null
''')
        with open(os.path.join('ESBDATA'+ f"//{account}.txt"), mode='w', encoding= "utf-8-sig") as file:
            file.write(f'''
-- ESBDATA
select t1.t24_trans_no trans_no, t1.field_102 as tai_khoan_doi_ung, t1.field_48 as ten_doi_ung
, srcbnk.name as ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_102 <> '{account}'
and t1.t24_trans_no is not null
union all
select t1.t24_trans_no trans_no, t1.field_103 tai_khoan_doi_ung, t1.field_120 ten_doi_ung, dstbnk.NAME ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_103 <> '{account}'
and t1.t24_trans_no is not null
union all
select t2.trans_no, t1.debit_account_no as tai_khoan_doi_ung, da.t_short_title as ten_doi_ung
, 'Ngân hàng TMCP Phương Đông (OCB)' as ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.debit_account_no <> '{account}'
union all
select t2.trans_no, t1.credit_account as tai_khoan_doi_ung
, ca.t_short_title as ten_doi_ung, 'Ngân hàng TMCP Phương Đông (OCB)' ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.credit_account <> '{account}'
''')
    if len(value.split(",")) >= 1000:
        a = value.split(",")
        sl_file = (count // 1000) + 1
        sl_row = (count // sl_file)  + 1
        start = 1
        start_index = 0
        end_index = sl_row
        while start <= sl_file:
            new_list_a = a[start_index:end_index]
            list_ft = ','.join(new_list_a)
            with open(os.path.join('OCBDWP'+ f"//{account}_{start}.txt"), mode='w', encoding= "utf-8-sig") as file:
                file.write(f'''
-- OCBDWP
select ft_no trans_no
, ben_acct_no tai_khoan_doi_ung
, ben_cust_name ten_doi_ung
, ben_bank_name ngan_hang_doi_ung
from ocbdwp.v_cust_acct_stmt_ben
where account = '{account}'
and ft_no in (
{list_ft}
)
and ben_acct_no is not null
''')
            with open(os.path.join('ESBDATA'+ f"//{account}_{start}.txt"), mode='w', encoding= "utf-8-sig") as file:
                file.write(f'''
-- ESBDATA
select t1.t24_trans_no trans_no, t1.field_102 as tai_khoan_doi_ung, t1.field_48 as ten_doi_ung
, srcbnk.name as ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_102 <> '{account}'
and t1.t24_trans_no is not null
union all
select t1.t24_trans_no trans_no, t1.field_103 tai_khoan_doi_ung, t1.field_120 ten_doi_ung, dstbnk.NAME ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_103 <> '{account}'
and t1.t24_trans_no is not null
union all
select t2.trans_no, t1.debit_account_no as tai_khoan_doi_ung, da.t_short_title as ten_doi_ung
, 'Ngân hàng TMCP Phương Đông (OCB)' as ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.debit_account_no <> '{account}'
union all
select t2.trans_no, t1.credit_account as tai_khoan_doi_ung
, ca.t_short_title as ten_doi_ung, 'Ngân hàng TMCP Phương Đông (OCB)' ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.credit_account <> '{account}'
''')
            start_index = end_index
            end_index = end_index + sl_row
            start +=1
# sql.close()
file.close()

# # Data Full

In [ ]:
# Import data
data_full = import_data('fileDataFull')

# Remove duplicate
data_full.drop_duplicates('TRANS_NO', inplace=True)

# Merge with file data_supplied
data_supplied = pd.merge(data_supplied, data_full, how="left", left_on="ID", right_on="TRANS_NO")

# Filter FT not value
df = data_supplied[data_supplied['TRANS_NO'].isna()]



# ____

In [53]:
df = pd.read_excel(r"D:\python\bank-Statement\check-Statements-File\statement_Special\#392109\#392109_full.xlsx", dtype=object)

def processFT(x): #Take first 12 character if is FT

    if x.startswith("FT"):
        x = x[:12]
        return x
    else:
        return x

df['ID'] = df['ID'].apply(processFT)

df.drop_duplicates('ID', inplace=True)


# DF FT

In [54]:
df_FT = pd.read_excel(r"D:\python\bank-Statement\check-Statements-File\statement_Special\#392109\data_FT.xlsx", dtype=object)

# Remove duplicate
df_FT.drop_duplicates('TRANS_NO', inplace=True)

df_result_1 = pd.merge(df, df_FT, how='left', left_on='ID', right_on='TRANS_NO')

df_result_1.to_excel(r"D:\python\bank-Statement\check-Statements-File\statement_Special\#392109\df_result_1.xlsx", index=False)

# df_result_2[df_result_2['TRANS_NO'].isna()]

# DF FT EXTRA

In [55]:
df_FT_extra = pd.read_excel(r"D:\python\bank-Statement\check-Statements-File\statement_Special\#392109\data_FT_extra.xlsx", dtype=object)


def processFT(x): #Take first 12 character if is FT

    if x.startswith("FT"):
        x = x[:12]
        return x
    else:
        return x

df_FT_extra['FT_NO'] = df_FT_extra['FT_NO'].apply(processFT)

df_FT_extra.drop_duplicates('FT_NO', inplace=True)

df_result_2 = pd.merge(df_result_1, df_FT_extra, how='left', left_on='ID', right_on='FT_NO')

In [56]:
for index, row in df_result_2.iterrows():
    if row['FT_NO'] is not np.NaN:
        if row['DEBIT_ACCOUNT'] == row['TAI_KHOAN_DOI_UNG']:
            row['DEBIT_ACCOUNT'] = ''
        else:
            row['CREDIT_ACCOUNT'] = ''

df_result_2['TAI_KHOAN_DOI_UNG'] = df_result_2['TAI_KHOAN_DOI_UNG'].fillna('') + df_result_2['DEBIT_ACCOUNT'].fillna('') + df_result_2['CREDIT_ACCOUNT'].fillna('')

col_remove = ['FT_NO','DEBIT_ACCOUNT', 'CREDIT_ACCOUNT']
df_result_2 = df_result_2.drop(col_remove, axis=1)

In [52]:
# df_result_2.to_excel(r"D:\python\bank-Statement\check-Statements-File\statement_Special\#392109\df_result_2.xlsx", index=False)


# Check Account

In [58]:
df_result_2.columns

Index(['DATA_DATE', 'MACN', 'TENCN', 'ID', 'TKNO', 'CIF_TKNO', 'TEN_TKNO',
       'LOAI_GIAYTO_KH_TKNO', 'MA_GIAYTO_KH_TKNO', 'NGOAITE_TKNO',
       'SOTIEN_NT_TKNO', 'SOTIEN_QD_TKNO', 'TKCO', 'CIF_TKCO', 'TEN_TKCO',
       'NGOAITE_TKCO', 'LOAI_GIAYTO_KH_TKNO.1', 'MA_GIAYTO_KH_TKNO.1',
       'SOTIEN_NT_TKCO', 'SOTIEN_QD_TKCO', 'NOIDUNG', 'TRANS_NO',
       'TAI_KHOAN_DOI_UNG', 'TEN_DOI_UNG', 'NGAN_HANG_DOI_UNG',
       'T_TRANSACTION_TYPE', 'DEBIT_ACCOUNT_NAME', 'BENEFICIARY',
       'BENEFICIARY_ADDRESS'],
      dtype='object')

In [80]:
# Collect a

dict_account = {}
for index, row in df_result_2.iterrows():
    if row['TKNO'] not in dict_account:
        dict_account[row['TKNO']] = row['TEN_TKNO']

    if row['TKCO'] not in dict_account:
        dict_account[row['TKCO']] = row['TEN_TKCO']
        
    if row['TEN_DOI_UNG'] is not np.NaN and row['TEN_DOI_UNG'] not in dict_account:
        dict_account[row['TAI_KHOAN_DOI_UNG']] = row['TEN_DOI_UNG']



In [82]:
# Check account column TEN_DOI_UNG

account_by_year = {}
for index, row in df_result_2.iterrows():
    if row['TEN_DOI_UNG'] is np.NaN:
        if row['TAI_KHOAN_DOI_UNG'] in dict_account:
            row['TEN_DOI_UNG'] = dict_account[row['TAI_KHOAN_DOI_UNG']]
        else:
            if row['DATA_DATE'][:4] in account_by_year:
                account_by_year[row['DATA_DATE'][:4]] = account_by_year[row['DATA_DATE'][:4]] + f",'{row['TAI_KHOAN_DOI_UNG']}'"
            else:
                account_by_year[row['DATA_DATE'][:4]] = f"'{row['TAI_KHOAN_DOI_UNG']}'"
print(account_by_year)

{'2022': "'0999999686868','070004668713','5503215056690','4509205023131','070004668713','060249326081','6016082003333','070113849401','4509205023131','070004668713','0111000338569','0001454495291','19038303013014','1013774354','603057150700001','102005123427','18072567','070133973379','0971145013','0111000353128','0000105056789','611094469900001','0841000073961','19036060845021','0181002437385','0111000187557','1629999999','9704229204515980192','1800205123578','7712205138984','105006916734','070066706867','7290737','7712205138984','102875517131','0531002581679','1019368349','334345566789','0833838178','12376987','102005123427','1019368349','0000105056789','7712222293939','7712205033667','070077954587','105006916734','104871241897','6688883979','0983586586','7712205101354','1015320735','0983586586','1027743256','107006610478','1890205162437','070101122997','1660566666','9017041007857','0111000628498','1028707725','070119504031','070101122997','7508205152015','7712205033667','07001011985

In [79]:
# Export file SQL select account 2022, 2023
# Need to add more some years than older


for year, accounts in account_by_year.items():

    count = len(accounts.split(","))
    
    if len(accounts.split(",")) < 1000:
        with open(os.path.join('sqlAccount'+ f"//{year}.txt"), mode='w', encoding= "utf-8-sig") as file:
            file.write(f'''
-- OCBDWP
SELECT DISTINCT id,
  t_short_title
FROM  ocbdw.t24_account
WHERE id IN ({accounts})
''')
    if len(accounts.split(",")) >= 1000:
        a = accounts.split(",")
        sl_file = (count // 1000) + 1
        sl_row = (count // sl_file)  + 1
        start = 1
        start_index = 0
        end_index = sl_row
        while start <= sl_file:
            new_list_a = a[start_index:end_index]
            list_ft = ','.join(new_list_a)
            with open(os.path.join('OCBDWP'+ f"//{year}_{start}.txt"), mode='w', encoding= "utf-8-sig") as file:
                file.write(f'''
-- OCBDWP
SELECT DISTINCT id,
  t_short_title
FROM  ocbdw.t24_account
WHERE id IN ({accounts})
''')

            start_index = end_index
            end_index = end_index + sl_row
            start +=1
# sql.close()
file.close()

# ------

In [44]:
folder = r'fileStatements'
listDataframe = []

for file in os.listdir(folder):
    df = pd.read_excel(os.path.join(folder, file), dtype=object)
    listDataframe.append(df)
df_full = pd.concat(listDataframe)

In [3]:
df_full = pd.read_excel(r'#392109/#392109_full.xlsx', dtype=object)

In [4]:
df_full.columns

Index(['DATA_DATE', 'MACN', 'TENCN', 'ID', 'TKNO', 'CIF_TKNO', 'TEN_TKNO',
       'LOAI_GIAYTO_KH_TKNO', 'MA_GIAYTO_KH_TKNO', 'NGOAITE_TKNO',
       'SOTIEN_NT_TKNO', 'SOTIEN_QD_TKNO', 'TKCO', 'CIF_TKCO', 'TEN_TKCO',
       'NGOAITE_TKCO', 'LOAI_GIAYTO_KH_TKNO.1', 'MA_GIAYTO_KH_TKNO.1',
       'SOTIEN_NT_TKCO', 'SOTIEN_QD_TKCO', 'NOIDUNG'],
      dtype='object')

In [6]:
df_full.drop_duplicates('ID', inplace=True)

In [15]:
for index, row in df_full.iterrows():

    if not row['TKNO'].startswith('FT'):
        print(index)
        print(row['TKNO'])
        break

0
0044100000052002


In [19]:
dict_data ={}


for index, row in df_full.iterrows():

    if row['ID'].startswith('FT'):
        if row['TKNO'] is np.NaN or row['TKNO'].startswith(('VND','PL')): 

            if row['TKCO'] in dict_data:
                dict_data[row['TKCO']] = dict_data[row['TKCO']] + f",'{row['ID'][:12]}'"
    
            else:
                dict_data[row['TKCO']] = f"'{row['ID'][:12]}'"
        else:

            if row['TKNO'] in dict_data:
                dict_data[row['TKNO']] = dict_data[row['TKNO']] + f",'{row['ID'][:12]}'"
            else:
                dict_data[row['TKNO']] = f"'{row['ID'][:12]}'"


# dict_data
for key, value in dict_data.items():
    account = key
    list_ft = value
    count = len(value.split(","))
    if len(value.split(",")) < 1000:
    # account =key
    # list_ft = value
        with open(os.path.join('OCBDWP'+ f"//{account}.txt"), mode='w', encoding= "utf-8-sig") as file:
            file.write(f'''
-- OCBDWP
select ft_no trans_no
, ben_acct_no tai_khoan_doi_ung
, ben_cust_name ten_doi_ung
, ben_bank_name ngan_hang_doi_ung
from ocbdwp.v_cust_acct_stmt_ben
where account = '{account}'
and ft_no in (
{list_ft}
)
and ben_acct_no is not null
''')
        with open(os.path.join('ESBDATA'+ f"//{account}.txt"), mode='w', encoding= "utf-8-sig") as file:
            file.write(f'''
-- ESBDATA
select t1.t24_trans_no trans_no, t1.field_102 as tai_khoan_doi_ung, t1.field_48 as ten_doi_ung
, srcbnk.name as ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_102 <> '{account}'
and t1.t24_trans_no is not null
union all
select t1.t24_trans_no trans_no, t1.field_103 tai_khoan_doi_ung, t1.field_120 ten_doi_ung, dstbnk.NAME ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_103 <> '{account}'
and t1.t24_trans_no is not null
union all
select t2.trans_no, t1.debit_account_no as tai_khoan_doi_ung, da.t_short_title as ten_doi_ung
, 'Ngân hàng TMCP Phương Đông (OCB)' as ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.debit_account_no <> '{account}'
union all
select t2.trans_no, t1.credit_account as tai_khoan_doi_ung
, ca.t_short_title as ten_doi_ung, 'Ngân hàng TMCP Phương Đông (OCB)' ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.credit_account <> '{account}'
''')
    if len(value.split(",")) >= 1000:
        a = value.split(",")
        sl_file = (count // 1000) + 1
        sl_row = (count // sl_file)  + 1
        start = 1
        start_index = 0
        end_index = sl_row
        while start <= sl_file:
            new_list_a = a[start_index:end_index]
            list_ft = ','.join(new_list_a)
            with open(os.path.join('OCBDWP'+ f"//{account}_{start}.txt"), mode='w', encoding= "utf-8-sig") as file:
                file.write(f'''
-- OCBDWP
select ft_no trans_no
, ben_acct_no tai_khoan_doi_ung
, ben_cust_name ten_doi_ung
, ben_bank_name ngan_hang_doi_ung
from ocbdwp.v_cust_acct_stmt_ben
where account = '{account}'
and ft_no in (
{list_ft}
)
and ben_acct_no is not null
''')
            with open(os.path.join('ESBDATA'+ f"//{account}_{start}.txt"), mode='w', encoding= "utf-8-sig") as file:
                file.write(f'''
-- ESBDATA
select t1.t24_trans_no trans_no, t1.field_102 as tai_khoan_doi_ung, t1.field_48 as ten_doi_ung
, srcbnk.name as ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_102 <> '{account}'
and t1.t24_trans_no is not null
union all
select t1.t24_trans_no trans_no, t1.field_103 tai_khoan_doi_ung, t1.field_120 ten_doi_ung, dstbnk.NAME ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_103 <> '{account}'
and t1.t24_trans_no is not null
union all
select t2.trans_no, t1.debit_account_no as tai_khoan_doi_ung, da.t_short_title as ten_doi_ung
, 'Ngân hàng TMCP Phương Đông (OCB)' as ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.debit_account_no <> '{account}'
union all
select t2.trans_no, t1.credit_account as tai_khoan_doi_ung
, ca.t_short_title as ten_doi_ung, 'Ngân hàng TMCP Phương Đông (OCB)' ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.credit_account <> '{account}'
''')
            start_index = end_index
            end_index = end_index + sl_row
            start +=1
# sql.close()
file.close()

In [33]:
df_data = pd.read_excel('data_FT.xlsx', dtype=object)

In [37]:
df_data.drop_duplicates('TRANS_NO', inplace=True)

In [ ]:
df_data['TRANS_NO'].value_counts()

In [39]:
df_data[df_data['TRANS_NO'] == 'FT211557PWKB']

,TRANS_NO,TAI_KHOAN_DOI_UNG,TEN_DOI_UNG,NGAN_HANG_DOI_UNG
829,FT211557PWKB,2500201013004,UB MAT TRAN TO QUOC TINH BAC GIANG,Ngân hàng Nông nghiệp và Phát triển Nông thôn ...


# Work

In [170]:
df = pd.read_excel('#390959\TK ĐỐI UNG _924890.xlsx', dtype=object)
df_data = pd.read_excel('data_FT.xlsx', dtype=object)


# df['FT_NO'] = df['FT_NO'].str[:12]
def processFT(x): #Take first 12 character if is FT

    if x.startswith("FT"):
        x = x[:12]
        return x
    else:
        return x
df['Số giao dịch'] = df['Số giao dịch'].apply(processFT)


df_merge = pd.merge(df, df_data, how='left', left_on='Số giao dịch', right_on='TRANS_NO')
df_merge = df_merge.astype(object)

df_old_years = pd.read_excel('data_old_years.xlsx', dtype=object)
df_old_years['FT_NO'] = df_old_years['FT_NO'].str[:12]

df_merge_1 = pd.merge(df_merge, df_old_years, how='left', left_on='Số giao dịch', right_on='FT_NO')
# df_merge_1.to_excel('TK ĐỐI UNG _924890.xlsx', index=False)

# df_merge_1 = df_merge_1[~df_merge_1['FT_NO'].isna()]
# df_2021.reset_index(inplace=True)

for index, row in df_merge_1.iterrows():
    if row['FT_NO'] is not np.NaN:
        if row['DEBIT_ACCOUNT'] == '0031100001520008':
            row['DEBIT_ACCOUNT'] = ''
        else:
            row['CREDIT_ACCOUNT'] = ''

df_merge_1['TAI_KHOAN_DOI_UNG'] = df_merge_1['TAI_KHOAN_DOI_UNG'].fillna('') + df_merge_1['DEBIT_ACCOUNT'].fillna('') + df_merge_1['CREDIT_ACCOUNT'].fillna('')

col_remove = ['FT_NO','DEBIT_ACCOUNT', 'CREDIT_ACCOUNT']
df_merge_1 = df_merge_1.drop(col_remove, axis=1)

#Account
df_account = pd.read_excel('2021_2022-2023_account_oldyear.xlsx', dtype=object)
df_account.drop_duplicates('ID', inplace=True)

# df_acc =pd.DataFrame(dict_temp.items(), columns=['Account', 'Name'])
dict_account = {}
for index, row in df_account.iterrows():
    dict_account[row['ID']] = row['T_SHORT_TITLE']


for index, row in df_merge_1.iterrows():

    if row['TAI_KHOAN_DOI_UNG'] in dict_account:
        row['TEN_DOI_UNG'] = dict_account[row['TAI_KHOAN_DOI_UNG']]


df_merge_1.to_excel('TK ĐỐI UNG _924890.xlsx', index=False)
# df_merge_1.head(10)


In [ ]:
df_account = pd.read_excel('2021_2022-2023_account_oldyear.xlsx', dtype=object)

df_account['ID'].value_counts()

In [156]:

#Account
df_account = pd.read_excel('2021_2022-2023_account_oldyear.xlsx', dtype=object)

# df_acc =pd.DataFrame(dict_temp.items(), columns=['Account', 'Name'])
dict_account = {}
for index, row in df_account.iterrows():
    dict_account[row['ID']] = row['T_SHORT_TITLE']
print(dict_account)

{'0906751644': 'LE THI KHUNG', 'VND1384500000001': 'LIEN HANG SAI LAM CI-TAD DEN', '0036100048256008': 'NGUYEN QUOC HOA', '0144102066668888': 'NGO HUY PHUC', 'VND1252011000001': 'PTNAPAS-GD ATM, IBT', 'VND1754210010001': 'TREO SAI LECH CHIEU THU HUONG', '0001500000006005': 'CITAD-TGTT TAI SGD NHNN HANOI(VND', '0020100013113007': 'PHAN THI THU HONG', '0908594453': 'NGUYEN THI KIM HUONG', 'VND1384500020001': 'LIEN HANG SAI LAM CI-TAD DI', '0001500000289007': 'HOI SO - TGTT TAI BIDV B.THANH VND', '0037100017728002': 'TRIEU THI TY', '0065100005599007': 'CTY CO  PHAN IBS', '0365707183': 'PHAN HONG DU', '0100100029674002': 'TRAN THI THU HUONG', 'VND1384700020001': 'SAI LAM BIDV - VND', '0026100002657007': 'NGUYEN THI YEN NHI'}


In [93]:
df_2021['TAI_KHOAN_DOI_UNG'] = df_2021['DEBIT_ACCOUNT'] + df_2021['CREDIT_ACCOUNT']

In [95]:
df_2021['TAI_KHOAN_DOI_UNG'].unique()

array(['0001500000006005', 'VND1754210010001', '0001500000289007',
       'PL52193', 'PL52002', 'VND1384500000001', '0100100029674002',
       'PL52022', '0026100002657007', 'VND1252011000001',
       'VND1384700020001'], dtype=object)

In [96]:
df_2022_1 = pd.read_excel('SAOKE_20220101_20220429.xlsx', dtype=object)
df_2022_2 = pd.read_excel('SAOKE_20220501_20230328.xlsx', dtype=object)
df_2022 = pd.concat([df_2022_1,df_2022_2])
df_2022.columns

Index(['STT/ No', 'STMT No', 'Ngay/ Date', 'So Tai khoan/Acc Number',
       'Ten Tai khoan/Acc Name', 'So CIF/ CIF Number', 'Loai tien/Currency',
       'FT', 'Noi Dung/Content', 'Ps giam nguyen te(No)/ Debit',
       'Ps giam quy doi(No)/ Debit', 'Ps tang nguyen te(Co)/ Credit',
       'Ps tang quy doi(Co)/ Credit', 'T_OPEN_CLEARED_BAL', 'TRANS_NO',
       'TAI_KHOAN_DOI_UNG', 'TEN_DOI_UNG', 'NGAN_HANG_DOI_UNG', 'FT_NO',
       'DEBIT_ACCOUNT', 'CREDIT_ACCOUNT', 'T_TRANSACTION_TYPE'],
      dtype='object')

In [97]:
df_2022 = df_2022[~df_2022['FT_NO'].isna()]
df_2022.reset_index(inplace=True)

In [ ]:
# df_2022.reset_index(inplace=True)
for index, row in df_2022.iterrows():
    if row['DEBIT_ACCOUNT'] == row['So Tai khoan/Acc Number']:
        df_2022.loc[index,'DEBIT_ACCOUNT'] = ''
    else:
        df_2022.loc[index,'CREDIT_ACCOUNT'] = ''

In [99]:
df_2022['TAI_KHOAN_DOI_UNG'] = df_2022['DEBIT_ACCOUNT'] + df_2022['CREDIT_ACCOUNT']

In [ ]:
df_2022['TAI_KHOAN_DOI_UNG'].unique()

In [42]:
import pandas as pd
import xlwings as xw

# create a sample DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie', 'David'],
        'Age': [25, 30, 35, 40],
        'Salary': [50000, 60000, 70000, 1.001E+14]}
df = pd.DataFrame(data)

# create an Excel workbook
wb = xw.Book()

# select the first sheet
sheet = wb.sheets[0]

# write the DataFrame to the worksheet
sheet.range('A1').options(index=False, header=True).value = df

# set the number format for the Salary column
salary_col = sheet.range('C2:C5')
salary_col.number_format = '#,##0.00'

# save the workbook
wb.save('data.xlsx')

# close the workbook
wb.close()


In [9]:
import pandas as pd

# create the first table
df1 = pd.DataFrame({'key': ['A', 'B', 'C', 'D'], 'value1': [1, 2, 3, 4]})

# create the second table
df2 = pd.DataFrame({'id': ['A', 'B', 'E', 'F'], 'value2': [5, 6, 7, 8]})

# merge the two tables based on the 'key' column
merged_df = pd.merge(df1, df2, how='left' , left_on='key', right_on='id')

print(merged_df)


  key  value1   id  value2
0   A       1    A     5.0
1   B       2    B     6.0
2   C       3  NaN     NaN
3   D       4  NaN     NaN


In [11]:
df1 = pd.read_excel(r"D:\python\bank-Statement\check-Statements-File\statement_Special\yc384052_saoke_dsCBNV.xlsx")
df2 = pd.read_excel(r"D:\python\bank-Statement\check-Statements-File\statement_Special\yc384052_saoke_dsKH.xlsx")

df_data = pd.read_excel(r"D:\python\tools\file\master.xlsx")

In [14]:
df_data= df_data.drop_duplicates('TRANS_NO')
df1= df1.drop_duplicates('ID')

In [13]:
df_data['TRANS_NO'].value_counts()

FT22006G2NXC    1
FT223143CXDJ    1
FT2225598GH2    1
FT22273RJVLQ    1
FT222857TNGH    1
               ..
FT23017Y9GDX    1
FT22357ZK2BX    1
FT230279J28M    1
FT23027KXZ15    1
FT222381DM3Y    1
Name: TRANS_NO, Length: 9918, dtype: int64

In [16]:
df1['ID'].value_counts()

FT220043WQJC;1    1
FT221822KTX7;1    1
FT222909VVJD;1    1
FT22136FGFNY;1    1
FT230652KYM8;1    1
                 ..
FT22343Y8VT9;1    1
FT22248GNXR4;1    1
FT22153N8B2W;1    1
FT23067W70S9;1    1
TT22197CX03P;1    1
Name: ID, Length: 8853, dtype: int64

In [77]:
df_data[df_data['TRANS_NO']=='FT2230195JB3']

,TRANS_NO,TAI_KHOAN_DOI_UNG,TEN_DOI_UNG,NGAN_HANG_DOI_UNG
4701,FT2230195JB3,8017041157136,PHAN CHU MINH CHINH,Ngân hàng TMCP Bản Việt (Vietcapital Bank)


In [48]:
print(len(df_data['TRANS_NO'].unique()))

617


In [39]:
df = pd.concat([df1, df_2])

In [43]:
df = df.drop_duplicates('ID')

In [44]:
df['ID'].value_counts()

FT220043WQJC;1    1
FT22042W75NW;1    1
FT22043BMLXV;1    1
FT2204384J92;1    1
FT22043GS6T3;1    1
                 ..
FT22217NNPV2;1    1
FT23017D1J9P;1    1
FT22358YS1BT;1    1
FT222633Z67L;1    1
TT222298YNB9;1    1
Name: ID, Length: 14665, dtype: int64

14665


In [41]:
df1 = pd.read_excel('yc383517_saoke_dsNV_2019.xlsx')
df2 = pd.read_excel('yc383517_sk_dsKH_2019.xlsx')

df = pd.concat([df1, df2])

In [42]:
df['ID'].value_counts()

FT1915162TNT;1    1
TT19340LN4YB;1    1
TT19183ZB6SW;1    1
TT190289V5B4;1    1
TT19007ZK50C;1    1
                 ..
FT19304XVMFS;1    1
FT1911489P61;1    1
FT19357BYRV9;1    1
FT19277S5XBB;1    1
TT19353VW5PY;1    1
Name: ID, Length: 1416, dtype: int64

# SQL

# Open file before merge file

In [44]:
df1 = pd.read_excel('yc383517_saoke_dsNV_2019.xlsx')
df2 = pd.read_excel('yc383517_sk_dsKH_2019.xlsx')

df = pd.concat([df1, df2])

df['ID'] = df['ID'].str[:12]
df = df.drop_duplicates('ID')
df_data = pd.read_excel('data_2019.xlsx')
df_data = df_data.drop_duplicates('TRANS_NO')

# Merge file

In [45]:
df['ID'] = df['ID'].str[:12]
# df2['ID'] = df2['ID'].str[:12]
result = pd.merge(df,df_data, how="left", left_on='ID', right_on='TRANS_NO')
# result = pd.merge(result,df_extra, how="left", left_on='ID', right_on='FT_NO')



result.to_excel((r"2019.xlsx"), index=False)

In [46]:
result['TRANS_NO'] is np.NaN

False

In [25]:
df1 = pd.read_excel('df1.xlsx')
df2 = pd.read_excel('df2.xlsx')
df = pd.concat([df1,df2])

In [22]:
df1['DATA_DATE'] = df1['DATA_DATE'].astype(str)
df2['DATA_DATE'] = df2['DATA_DATE'].astype(str)

In [38]:
df2['DATA_DATE'].str[:4].value_counts()

2022    4879
2023     942
Name: DATA_DATE, dtype: int64

In [61]:
df_master = pd.read_excel('df1.xlsx')

In [26]:
filt_TK_Null = df['TRANS_NO'].isna()

df = df.loc[filt_TK_Null]

# df_master.to_excel((r"D:\python\bank-Statement\check-Statements-File\oldYear\master.xlsx"), index=False)

In [28]:
df

,,DATA_DATE,MACN,TENCN,ID,TKNO,CIF_TKNO,TEN_TKNO,NGOAITE_TKNO,SOTIEN_NT_TKNO,...,NGOAITE_TKCO,SOTIEN_NT_TKCO,SOTIEN_QD_TKCO,NOIDUNG,INPUTTER,AUTHORISER,TRANS_NO,TAI_KHOAN_DOI_UNG,TEN_DOI_UNG,NGAN_HANG_DOI_UNG
1,2.0,20220104,VN0010024,PGD NGUYEN TRI PHUONG,FT220048H6Y0,0100100004845007,814344.0,HOANG THI NGOC HUYEN,VND,10000000,...,VND,10000000,10000000,HUYEN chuyen tien cho Cuc,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,20220104,VN0010001,HOI SO,FT21365KYB7T,0001100001432007,814344.0,HOANG THI NGOC HUYEN,VND,3868,...,VND,3868,3868,Overdraft Repayment,NaN,NaN,NaN,NaN,NaN,NaN
7,8.0,20220125,VN0010001,HOI SO,FT22025369LH,VND1755300100001,NaN,CAC KHOAN PT CBNV-LUONG TAP TRUNG,VND,500000,...,VND,500000,500000,CONG DOAN OCB TANG QUA NHAN DIP TET::NHAM DAN ...,NaN,NaN,NaN,NaN,NaN,NaN
10,11.0,20220106,VN0010021,PGD PHAM VAN HAI,FT22006N2Q32,0024109959999999,2218992.0,TONG TRUC THANH,VND,50000,...,VND,50000,50000,NAP TIEN DIEN THOAI 0904421782 – ME::NH GIA 50000,NaN,NaN,NaN,NaN,NaN,NaN
12,13.0,20220108,VN0010024,PGD NGUYEN TRI PHUONG,FT220084Z177,0024101123456787,1860246.0,NGUYEN TAN DAT,VND,559634,...,VND,559634,559634,Thanh toan hoa don Ma KH PE07000107::262,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5816,NaN,20221001,VN0010024,PGD NGUYEN TRI PHUONG,TT22274Y6CY7,0903346236,7653906.0,NGUYEN HUYNH THAI HUY,VND,NaN,...,VND,0,200000000,NGUYEN HUYNH THAI HUY CMND 0720790!!04994 NGAY...,T0240014,T240507,NaN,NaN,NaN,NaN
5817,NaN,20230216,VN0010024,PGD NGUYEN TRI PHUONG,TT230473KVNG,0024110003263003,813054.0,NGUYEN THI THU NGUYET,VND,NaN,...,VND,0,400000000,NGUYEN THI THU NGUYET-079183024536!!-15/08/202...,T0240014,T240509,NaN,NaN,NaN,NaN
5818,NaN,20220511,VN0010024,PGD NGUYEN TRI PHUONG,TT221313QJMP,0024100000922008,813054.0,NGUYEN THI THU NGUYET,VND,NaN,...,VND,0,30000000,NGUYEN THI THU NGUYET CMND 0791830!!24536 NGAY...,T0240014,T240343,NaN,NaN,NaN,NaN
5819,NaN,20220720,VN0013002,PGD BINH PHU,TT222019J0PD,0024107938389632,2329389.0,CHUNG CHI MINH,VND,NaN,...,VND,0,10000000,CHUNG CHI MINH CMND 079080009174 N!!GAYCAP 02/...,T3020536,T3020529,NaN,NaN,NaN,NaN


In [ ]:
df.to_excel('df.xlsx', index=False)

In [66]:
df_master['ID'].str.startswith('FT')

0       True
1       True
4       True
5       True
14      True
        ... 
9943     NaN
9944     NaN
9945     NaN
9946     NaN
9947     NaN
Name: ID, Length: 5326, dtype: object

In [67]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5326 entries, 0 to 9947
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0                      5326 non-null   int64  
 1   DATA_DATE          5326 non-null   int64  
 2   MACN               5326 non-null   object 
 3   TENCN              5326 non-null   object 
 4   ID                 2294 non-null   object 
 5   TKNO               5326 non-null   object 
 6   CIF_TKNO           4169 non-null   float64
 7   TEN_TKNO           5314 non-null   object 
 8   NGOAITE_TKNO       5326 non-null   object 
 9   SOTIEN_NT_TKNO     4375 non-null   object 
 10  SOTIEN_QD_TKNO     5326 non-null   object 
 11  TKCO               5326 non-null   object 
 12  CIF_TKCO           3431 non-null   float64
 13  TEN_TKCO           5313 non-null   object 
 14  NGOAITE_TKCO       5326 non-null   object 
 15  SOTIEN_NT_TKCO     5326 non-null   int64  
 16  SOTIEN_QD_TKCO     5326 

In [31]:
df = pd.read_excel(r"D:\python\bank-Statement\check-Statements-File\oldYear\master_account.xlsx")

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1974 entries, 0 to 1973
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  1974 non-null   object
 1   DEBIT_ACCOUNT       1974 non-null   object
 2   CREDIT_ACCOUNT      1974 non-null   object
 3   T_TRANSACTION_TYPE  1974 non-null   object
dtypes: object(4)
memory usage: 61.8+ KB


In [38]:
lenght = df['DEBIT_ACCOUNT'].unique()

print(len(lenght))

29


In [34]:
df.drop_duplicates('DEBIT_ACCOUNT')

,ID,DEBIT_ACCOUNT,CREDIT_ACCOUNT,T_TRANSACTION_TYPE
0,FT18009BN415;1,0008100007845007,0030100002402008,ACPB
1,FT18005LBPCZ;1,0008100003666003,PL52716,AC
2,FT18009ZRF7N;1,0008100006236007,0168160001052007,ACAM
9,FT180087CS41;1,VND1252001000001,0008100006236007,AC8S
17,FT180024KRWR;1,0008100004809002,0008100004658004,ACIB
19,FT1801800728;1,VND1384500000001,0008100004809002,AC
27,FT18017JYJL7;1,0008100001887002,0008100006236007,ACIB
30,FT18015QZCHJ;1,0117100000534008,0008100004809002,ACIB
43,FT18026KV1Y0;1,0001500000006005,0008100006236007,BIIB
46,FT1803145MP5;1,VND1750011790008,0008100004809002,ACLO
